In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from textblob import TextBlob
import csv,re,time

In [38]:
def get_platform(text,platform_value):
    blob = TextBlob(text)
    for word,tag in blob.tags:
        #print(word,tag,"\n")
        if tag=="NN":
            if word!="stated":
                platform_value=word
    Months=["January","February","March","April","May","June","July","August","September","October","November","December","january","february","march","april","may","june","july","august","september","october","november","december"]
    for i in blob.noun_phrases:
        if i not in Months:
            platform_value=i
    return platform_value

In [39]:
def get_links_from_page(writer:csv.writer):

    results=driver.find_elements(by=By.CSS_SELECTOR,value='article[class*="m-statement m-statement--is-medium m-statement"]')
    
    for result in results: # for each story div
        
        date,platform,text,name,truth='NA','NA','NA','NA','NA'
       
        try: # try to get the name
            name_box=result.find_element(by=By.CSS_SELECTOR,value='a[class="m-statement__name"]')
            name=name_box.text

        except NoSuchElementException as e:
            print('could not extract name')
            
        try: 
            data_box=result.find_element(by=By.CSS_SELECTOR,value='div[class="m-statement__desc"]')
            fact_date=data_box.text
            #print(fact_date)
            date_object = re.findall(r'(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{1,2}, )?\d{2,4}',fact_date)
            date = ' '.join(map(str, date_object))
            #print(date)
            platform_value="No Platform"
            platform=get_platform(data_box.text,platform_value)
            #print(platform)

        except NoSuchElementException as e:
            print('could not extract date')
            
        try: 
            review_text_box=result.find_element(by=By.CSS_SELECTOR,value='div[class="m-statement__quote"]')
            text=review_text_box.text
            #print(text)

        except NoSuchElementException as e:
            print('could not extract text')
            
        try:
            truthfullness_box=result.find_element(by=By.CSS_SELECTOR,value='div[class="m-statement__meter"]')
            truth_data = truthfullness_box.find_element(by=By.CSS_SELECTOR,value='img[class="c-image__thumb"]')
            truth = truth_data.get_attribute('alt')

        except NoSuchElementException as e:
            print('could not extract truthfullness_category')
        
        #write a new row for this story
        writer.writerow([date,platform,text,name,truth])

In [40]:
def scrape(driver,url):
    delay:int = 10   # number of seconds to wait 

    driver.maximize_window()
    driver.get(url)  # visit the page

    #input() # wait until we can click on the cookie overlay

    page_count = 1  # keep track of page count

    # create a new csv writer for the review data
    fw=open('data.csv','w',encoding="utf-8")  # output file

    writer=csv.writer(fw,lineterminator='\n')   # create a csv writer for this file
    
    writer.writerow(['Date','Platform','Text','Person','Truthfulness Category'])

    while True: # keep going until there are no more pages

        print("Page No: ",page_count)   # print current page count

        page_count+=1   # increment page count

        get_links_from_page(writer)    # extract and write the review data from the current page
        
        try:
            # wait until the next button appears
            next_button = WebDriverWait(driver, delay).until(EC.element_to_be_clickable((By.LINK_TEXT, 'Next'))) 
            
            #if (page_count == 5):  #test case for running upto 5 page
            #    break
        
        except TimeoutException:
            break

        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(delay)
        
    fw.close()    

In [41]:
url = 'https://www.politifact.com/factchecks/list/?category=truth-o-meter&ruling='
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) 

In [42]:
scrape(driver,url)

Page No:  1
Page No:  2
Page No:  3
Page No:  4
Page No:  5
Page No:  6
Page No:  7
Page No:  8
Page No:  9
Page No:  10
Page No:  11
Page No:  12
Page No:  13
Page No:  14
Page No:  15
Page No:  16
Page No:  17
Page No:  18
Page No:  19
Page No:  20
Page No:  21
Page No:  22
Page No:  23
Page No:  24
Page No:  25
Page No:  26
Page No:  27
Page No:  28
Page No:  29
Page No:  30
Page No:  31
Page No:  32
Page No:  33
Page No:  34
Page No:  35
Page No:  36
Page No:  37
Page No:  38
Page No:  39
Page No:  40
Page No:  41
Page No:  42
Page No:  43
Page No:  44
Page No:  45
Page No:  46
Page No:  47
Page No:  48
Page No:  49
Page No:  50
Page No:  51
Page No:  52
Page No:  53
Page No:  54
Page No:  55
Page No:  56
Page No:  57
Page No:  58
Page No:  59
Page No:  60
Page No:  61
Page No:  62
Page No:  63
Page No:  64
Page No:  65
Page No:  66
Page No:  67
Page No:  68
Page No:  69
Page No:  70
Page No:  71
Page No:  72
Page No:  73
Page No:  74
Page No:  75
Page No:  76
Page No:  77
Page No: